# Cotação de FIIs

Script que extrai a cotação atualizada de cotas de fundos imobiliários escolhidas para um arquivo .csv.

In [ ]:
# Instalação das bibliotecas
#!pip install requests --quiet
#!pip install pandas --quiet
#!pip install beautifulsoup4 --quiet
#!pip install matplotlib

In [2]:
# Importando as bibliotecas
import datetime
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [7]:
# URL com a tabela de dados
url = "https://www.fundamentus.com.br/fii_resultado.php"

# Obtendo o conteúdo da página em formato de texto

# https://stackoverflow.com/questions/68259148/getting-404-error-for-certain-stocks-and-pages-on-yahoo-finance-python
headers = { 
    'User-Agent'      : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36', 
    'Accept'          : 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 
    'Accept-Language' : 'en-US,en;q=0.5',
    'DNT'             : '1', # Do Not Track Request Header 
    'Connection'      : 'close'
}
data = requests.get(url, headers=headers, timeout=5).text
soup = BeautifulSoup(data,"html.parser")

# Procurando a tabela da página
table = soup.find('table') # em html uma tabela é representada pela tag <table>

# Definindo dataframe
df = pd.DataFrame(columns=['Papel', 'Tipo', 'Segmento', 'Administradora', 'Cotação', 'DY (12M)', 'DY Atual', 'PVP',
                 'Liquidez diária', 'Valor Patrimonial', 'Valor de mercado','Vacância média',
                 'Último dividendo', 'Magic Number', 'Valor do Magic Number', 'FFO Yield', 
                 'Cap Rate', 'Qtd de imóveis', 'Preço do m²', 'Aluguel por m²'])

# Obtendo todas as linhas da tabela
for row in table.tbody.find_all('tr'): # em html uma linha da tabela é representada pela tag <tr>
    # Obtendo todas as colunas em cada linha
    columns = row.find_all('td')  # em html uma coluna da tabela é representada pela tag <td>
    if(columns != []):
        papel = columns[0].text.strip(' ')
        segmento = columns[1].text.strip(' ')
        cotacao = columns[2].text.strip(' ')
        ffo = columns[3].text.strip(' ')
        dy12 = columns[4].text.strip(' ')
        pvp = columns[5].text.strip(' ')
        vlrmercado = columns[6].text.strip(' ')
        liqdiaria = columns[7].text.strip(' ')
        qtdimoveis = columns[8].text.strip(' ')
        precom2 = columns[9].text.strip(' ')
        aluguelm2 = columns[10].text.strip(' ')
        caprate = columns[11].text.strip(' ')
        vacancia = columns[12].text.strip(' ')
        df = pd.concat([df, pd.DataFrame.from_records([{'Papel': papel,  'Segmento': segmento, 'Cotação': cotacao,
                                                        'FFO Yield': ffo, 'DY (12M)': dy12, 'PVP':pvp, 'Valor de mercado': vlrmercado,
                                                        'Liquidez diária': liqdiaria, 'Qtd de imóveis': qtdimoveis,
                                                        'Preço do m²': precom2, 'Aluguel por m²': aluguelm2,
                                                        'Cap Rate': caprate, 'Vacância média': vacancia}])], ignore_index=True)

# Lista de fundos desejados
listaFiltro = ['BRCR11', 'BTCI11', 'BTLG11', 'CPTS11', 'GARE11', 'HGBS11', 'HGLG11', 'HGRE11', 'HGRU11', 'JSRE11', 'KNCA11',
               'KNCR11', 'KNRI11', 'KNSC11', 'MALL11', 'MXRF11', 'SNAG11', 'URPR11', 'VGHF11', 'VGIA11', 'VINO11', 'VISC11',
               'XPCA11', 'XPLG11', 'XPML11']

df = df[df.Papel.isin(listaFiltro)]
df = df.reset_index(drop=True)

# Para buscar o valor do último dividendo
for item in listaFiltro:
    url="https://www.fundamentus.com.br/fii_proventos.php?papel="+item+"&tipo=2"
    data = requests.get(url, headers=headers, timeout=5).text
    soup = BeautifulSoup(data,"html.parser")
    table = soup.find('table') # em html uma tabela é representada pela tag <table>
    df2 = pd.DataFrame(columns=['Último dividendo'])
    for row in table.tbody.find_all('tr'): # em html uma linha da tabela é representada pela tag <tr>
        columns = row.find_all('td') # em html uma coluna da tabela é representada pela tag <td>
        if(columns != []):
            dividendo = columns[3].text.strip(' ')
            df2 = pd.concat([df2, pd.DataFrame.from_records([{'Último dividendo': dividendo}])], ignore_index=True)
    df['Último dividendo'].iloc[listaFiltro.index(item)]=df2.iloc[0,0]

# Para buscar o valor patrimonial
for item in listaFiltro:
    url="https://www.fundamentus.com.br/detalhes.php?papel="+item
    data = requests.get(url, headers=headers, timeout=5).text
    soup = BeautifulSoup(data,"html.parser")
    vlrpatrimonial = soup.findAll(True, {'class':['txt']})[75].get_text()
    df['Valor Patrimonial'].iloc[listaFiltro.index(item)] = vlrpatrimonial    

# Para buscar o nome da administradora do fundo, seu tipo e segmento
for item in listaFiltro:
    url="https://investidor10.com.br/fiis/"+item
    data = requests.get(url, headers=headers, timeout=5).text
    soup = BeautifulSoup(data,"html.parser")
    adm = soup.find('h2', class_="name-company").get_text()
    tipo = soup.findAll(True, {'class':['value']})[6].get_text()[34:-30]
    segmento = soup.findAll(True, {'class':['value']})[5].get_text()[34:-30]
    df['Administradora'].iloc[listaFiltro.index(item)] = adm
    df['Tipo'].iloc[listaFiltro.index(item)] = tipo
    df['Segmento'].iloc[listaFiltro.index(item)] = segmento

# Tratamento de dados
df['Cotação'] = df['Cotação'].replace({',': '.'}, regex=True)
df['Último dividendo'] = df['Último dividendo'].replace({',': '.'}, regex=True)
df['Cotação'] = df['Cotação'].astype(float)
df['Último dividendo'] = df['Último dividendo'].astype(float)
df['Magic Number'] = (df['Cotação'] / df['Último dividendo']).apply(np.ceil) 
df['Valor do Magic Number'] = (df['Magic Number'] * df['Cotação']).round(2)
df['DY Atual'] = ((((df['Último dividendo'] / df['Cotação'] + 1)**12) -1)* 100).round(2)

df['Cotação'] = df['Cotação'].astype(str)
df['Último dividendo'] = df['Último dividendo'].astype(str)
df['Valor do Magic Number'] = df['Valor do Magic Number'].astype(str)
df['DY Atual'] = df['DY Atual'].astype(str)
df['Magic Number'] = df['Magic Number'].astype(int)

df['Cotação'] = df['Cotação'].str.replace('.', ',')
df['Último dividendo'] = df['Último dividendo'].str.replace('.', ',')
df['Valor do Magic Number'] = df['Valor do Magic Number'].str.replace('.', ',')
df['DY Atual'] = df['DY Atual'].str.replace('.', ',')+"%"

'''# Salvar o arquivo em .csv
df.to_csv(r'C:/1/FII.csv', index = False, encoding = 'utf-8-sig')
print("Concluído às " + datetime.datetime.now().strftime("%H:%M:%S de %d/%m/%Y"))
'''
# Salvar o arquivo em .xlsx
df.to_excel(r'C:/1/FII.xlsx', index = False)
print("Concluído às " + datetime.datetime.now().strftime("%H:%M:%S de %d/%m/%Y"))

df

Concluído às 09:27:58 de 14/02/2025


,Papel,Tipo,Segmento,Administradora,Cotação,DY (12M),DY Atual,PVP,Liquidez diária,Valor Patrimonial,Valor de mercado,Vacância média,Último dividendo,Magic Number,Valor do Magic Number,FFO Yield,Cap Rate,Qtd de imóveis,Preço do m²,Aluguel por m²
0,BRCR11,Fundo de Tijolo,Híbrido,BTG PACTUAL CORPORATE OFFICE,"39,24","14,71%","15,71%","0,45",1.423.750,2.311.480.000,1.045.280.000,"4,71%","0,48",82,"3217,68","0,26%","17,00%",6,"9.338,66","1.587,88"
1,BTCI11,Fundo de papel,Títulos e Valores Mobiliários,FII BTG Pactual Fundo de CRI,"8,01","13,65%","14,35%","0,81",1.491.520,981.325.000,797.165.000,"0,00%","0,09",89,"712,89","13,70%","0,00%",0,"0,00","0,00"
2,BTLG11,Fundo de Tijolo,Logístico / Indústria / Galpões,BTG PACTUAL LOGÍSTICA,"94,11","9,83%","11,68%","0,91",8.618.300,4.450.090.000,4.070.180.000,"2,46%","0,87",109,"10257,99","6,70%","4,84%",19,"5.583,20","270,01"
3,CPTS11,Fundo de papel,Títulos e Valores Mobiliários,CAPITANIA SECURITIES II,"6,18","14,34%","16,69%","0,72",7.817.760,2.716.290.000,1.964.180.000,"0,00%","0,08",78,"482,04","9,53%","0,00%",0,"0,00","0,00"
4,GARE11,Fundo de Tijolo,Híbrido,GUARDIAN REAL ESTATE,"7,87","13,27%","12,9%","0,85",5.984.650,1.371.360.000,1.160.500.000,"0,00%","0,08",99,"779,13","6,59%","10,86%",4,"281,62","30,58"
5,HGBS11,Fundo de Tijolo,Shoppings / Varejo,HEDGE BRASIL SHOPPING,"174,98","11,31%","11,54%","0,81",2.834.100,2.736.590.000,2.227.460.000,"5,24%","1,6",110,"19247,8","9,29%","9,29%",15,"4.130,77","383,83"
6,HGLG11,Fundo de Tijolo,Logístico / Indústria / Galpões,CSHG LOGÍSTICA,"147,83","4,99%","9,3%","0,94",7.074.360,5.317.680.000,4.994.820.000,"6,43%","1,1",135,"19957,05","7,07%","7,38%",26,"3.149,01","232,27"
7,HGRE11,Fundo de Tijolo,Lajes Corporativas,CSHG REAL ESTATE FUNDO,"94,86","6,21%","10,33%","0,62",1.376.680,1.798.910.000,1.121.030.000,"26,34%","0,78",122,"11572,92","7,51%","11,14%",16,"6.003,27","668,85"
8,HGRU11,Fundo misto,Híbrido,CSHG RENDA URBANA,"110,2","5,42%","9,66%","0,88",5.313.560,2.307.520.000,2.028.390.000,"0,97%","0,85",130,"14326,0","8,67%","10,66%",68,"3.952,96","421,21"
9,JSRE11,Outro,Híbrido,JS REAL ESTATE MULTIGESTÃO - FII,"51,23","11,12%","11,84%","0,50",1.769.970,2.112.590.000,1.063.910.000,"7,05%","0,48",107,"5481,61","10,58%","12,21%",6,"9.119,22","1.113,81"
